In [66]:
""" Setting up libraries for project
    
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import folium as fm # Map Library: will only work with Jupyter since it needs a browser
from folium.plugins import FastMarkerCluster # To add map markers
from pyproj import Proj, transform
from geopy import geocoders
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

geolocator = Nominatim(user_agent="Traffic Data", timeout=3)
location = geolocator.geocode("175 5th Avenue NYC")

""" Settings for numpy and Pandas
"""
#Numpy:
np.set_printoptions(threshold = np.inf, linewidth = 500, suppress = True)

#Pandas
pd.set_option("display.max_rows"	, 	3000)
pd.set_option("display.max_columns"	, 	3000)
pd.set_option("display.width"		,	1000)
pd.set_option("display.float_format", 	lambda x: "%.3f' % x")





In [67]:
def get_longitude (row):
    if geolocator.geocode(str(row["Offense Street Name"]))!= None:
        return geolocator.geocode(str(row["Offense Street Name"])).longitude
    else:
        return None
        
def get_latitude (row):
    if geolocator.geocode(str(row["Offense Street Name"]))!= None:
        return geolocator.geocode(str(row["Offense Street Name"])).latitude
    else:
        return None
    

In [68]:
"""Parsing for the CSV, read in all data from CSV 
"""
def ParseFile(file_in, file_out, batchsize):
    """ 
    Recieve CSV file, parse data, then output new CSV organized with wanted data
    Parameter: file_in, file_out, batchsize (for larger CSVs if wanted >1mill entries)
    Return: Outputs new CSV with desired data, nothing returned from function
    """
    # Datafram for CSV
    d = pd.read_csv(file_in, delimiter = ',', header = 0, nrows = 0, dtype = str)
    dat_recent = pd.DataFrame(data = d)
    # The columns that we won't be using can be dropped
    dat_recent = dat_recent.drop(['School Zone', 'Construction Zone', 'Case Closed', 'Agency', 'Officer Code'], axis=1)

    i = 0
    not_done = True
    # Do in batches, if more than 1 million samples
    while not_done:

        print('processing batch', i, ', samples processed: ', i * batchsize)

        # load in batches of 1-million entries for processing per pass
        dat = pd.read_csv(file_in, delimiter = ',', header = 0, nrows = batchsize, skiprows = range(1, i*batchsize),
                          dtype = str)

        # Drop the columns that we don't need to save on space
        dat = dat.drop(['School Zone', 'Construction Zone', 'Case Closed', 'Agency', 'Officer Code'], axis=1)


        # if the batch has less than 1-million entries then we know that this is the last pass
        i+=1
        if len(dat) < batchsize:
            not_done = False

        # replace the emply fields with "0"
        dat = dat.replace(np.nan, '0')

        # Extract 2018 stuff and convert address to coordinates while storing into csv
        #print(location)
        dat = dat[dat['Offense Date'].str.contains('2018')]
        #dat["Latitude"]
        dat["Offense Street Name"] = dat["Offense Street Name"] + " Austin"
        #dat["Longitude"] = dat.apply (lambda row: get_longitude(row), axis=1)
        #dat["Latitude"] = dat.apply (lambda row: get_latitude(row), axis=1)
        dat.insert(0, "Longitude",dat.apply (lambda row: get_longitude(row), axis=1) )
        dat.insert(1, "Latitude",dat.apply (lambda row: get_latitude(row), axis=1) )
        dat.to_csv(file_out)
        #area = dat["Offense Street Name"]
        
        #for _, val in area.iteritems():
            #area = str(val)
            #print(area)
            #coord = geolocator.geocode(area)
            #print(coord)
            #if coord != None:
                #print(coord.latitude, coord.longitude)
        #location = geolocator.geocode(dat["Offense Street Name"], timeout=20)
        #dat["Coordinates"]
       

        #dat_recent = dat_recent.append(dat)


    #print(dat_recent)
    #dat_recent.to_csv(file_out)
    print('done, new .csv saved as', file_out)
    



In [69]:
"""Main Program
    Runs mapping library
"""

# To keep track of how long it takes
start_time = time.time()

# No need to run either if the csv's are already made
batchsize = 1000000
trim_data = True

if trim_data == True:
    #ParseFile('Municipal_Court_Caseload_Information_FY_2018.csv', 'Output2018.csv', batchsize)
    ParseFile("test.csv", "Output2018.csv", batchsize)
# Load in the cleaned data as its own dataframe to work with
print('Loading data...')
working_data = pd.read_csv('Output2018.csv', delimiter=',', header=0, dtype = object)

# Rename the first column which is duplicated upon loading the csv (csv saves the index and loading it in adds an index)
working_data = working_data.rename(columns = {'Unnamed: 0' : 'Index'})

print('Data loaded, producing figures and maps...')

# - - - - - - - - - - - - - - - - For the map: - - - - - - - - - - - - - - - -

# Take the coorinates as their own dataframe so it can be manipulated, no need to throw out the rows with bad
# coordinate data from the full data set since they may contain other useful info

cbatch = 40000 # Number of previous incidents to plot, anything over this will bog down the map
#coords = (working_data.loc[(len(working_data) - cbatch):, "Offense Street Name"]).astype(dtype = "str")
longitude = pd.to_numeric(working_data["Longitude"].dropna())
latitude = pd.to_numeric(working_data["Latitude"].dropna())



# # coords are in x/y and we want lat/long, this is from the pyproj documentation
# pm = '+proj=lcc +lat_1=34.03333333333333 +lat_2=35.46666666666667 +lat_0=33.5 +lon_0=-118 +x_0=2000000 ' \
#      '+y_0=500000.0000000002 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 +no_defs'

# # convert to lat/long
# x_in,y_in = coords['Latitude'].values, coords['Longitude'].values
# lat,long = transform(Proj(pm, preserve_units = True), Proj("+init=epsg:4326"), x_in,y_in)

AUS_coords = [30.267153,-97.7430608]
m = fm.Map(location=AUS_coords, zoom_start=10.5)

# # add map markers, plots as "long/lat" rather than "lat/long"
FastMarkerCluster(data=list(zip(latitude, longitude))).add_to(m)
display(m) 

processing batch 0 , samples processed:  0
done, new .csv saved as Output2018.csv
Loading data...
Data loaded, producing figures and maps...
